In [1]:
import warnings, requests, zipfile, io 
warnings.simplefilter('ignore')
import pandas as pd
import numpy as np
import os
import boto3
import sagemaker
from scipy.io import arff
from sklearn.model_selection import train_test_split
from sagemaker.image_uris import retrieve

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
# Specify the path to the ZIP file and the extraction directory
zip_file_path = 'food-twentieth-century-crop-statistics-1900-2017-xlsx.zip'  # Replace with the actual path to your ZIP file
extraction_path = './data'  # Specify the directory where you want to extract the contents

# Extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

# List the files in the extraction directory to see the extracted files
extracted_files = os.listdir(extraction_path)
print("Extracted files:", extracted_files)

# Assuming you want to read the first Excel file in the extraction directory
excel_file_path = os.path.join(extraction_path, extracted_files[0])
df = pd.read_excel(excel_file_path)

# Now, 'df' contains your data as a DataFrame that you can work with.

Extracted files: ['food-twentieth-century-crop-statistics-1900-2017-documentation.pdf', 'food-twentieth-century-crop-statistics-1900-2017-xlsx.xlsx']


ValueError: Excel file format cannot be determined, you must specify an engine manually.

In [4]:
# Load the Excel file
file = pd.ExcelFile("data/food-twentieth-century-crop-statistics-1900-2017-xlsx.xlsx")

# Read data from a specific sheet (e.g., 'CropStats')
sheet_name = 'CropStats'  # Replace with the name of the sheet you want to load
df = file.parse(sheet_name)

# Set the index to the first column and remove the index name
df = df.set_index(df.columns[0])
df.index.name = None

# Now, 'df' contains the data from the specified sheet with the index set as described.


In [5]:
df.head()

,Harvest_year,admin0,admin1,crop,hectares (ha),production (tonnes),year,yield(tonnes/ha),admin2,notes
0,1902,Austria,NaN,wheat,NaN,NaN,1902,1.31,NaN,NaN
1,1903,Austria,NaN,wheat,NaN,NaN,1903,1.47,NaN,NaN
2,1904,Austria,NaN,wheat,NaN,NaN,1904,1.27,NaN,NaN
3,1905,Austria,NaN,wheat,NaN,NaN,1905,1.33,NaN,NaN
4,1906,Austria,NaN,wheat,NaN,NaN,1906,1.28,NaN,NaN


In [6]:
#cleaning
df.drop(['admin2', 'notes', 'Harvest_year'], axis=1, inplace=True)
df.rename(columns = {'admin0': 'national', 'admin1': 'subnational', 'hectares (ha)': 'hectares_ha', 'production (tonnes)': 'production_tonnes', 'yield(tonnes/ha)': 'yield_tonnes_ha'}, inplace=True)
df.loc[df['subnational'].isna(), 'subnational'] = df['national']

mask = df['yield_tonnes_ha'].isna() & ~df['production_tonnes'].isna() & ~df['hectares_ha'].isna() & df['hectares_ha'] != 0
df.loc[mask, 'yield_tonnes_ha'] = df['production_tonnes'] / df['hectares_ha']
df.dropna(subset=['yield_tonnes_ha'], inplace=True)
len(df[~pd.isnull(df['hectares_ha']) & pd.isnull(df['production_tonnes']) & ~pd.isnull(df['yield_tonnes_ha'])])
# The mask is used here because of I did the same way as above it kept timing out
mask = df['production_tonnes'].isna() & ~df['yield_tonnes_ha'].isna() & ~df['hectares_ha'].isna()
df.loc[mask, 'production_tonnes'] = df['yield_tonnes_ha'] * df['hectares_ha']
df.dropna(subset=['production_tonnes'], inplace=True)
mask = df['hectares_ha'].isna() & ~df['yield_tonnes_ha'].isna() & ~df['production_tonnes'].isna()
df.loc[mask, 'hectares_ha'] = df['yield_tonnes_ha'] * df['production_tonnes']
df.dropna(subset=['hectares_ha'], inplace=True)
# The columns we just adapted just changed into objects, let's make them floats again
df['hectares_ha'] = df['hectares_ha'].astype(float)
df['production_tonnes'] = df['production_tonnes'].astype(float)
df['yield_tonnes_ha'] = df['yield_tonnes_ha'].astype(float)
df['log_yield'] = np.log1p(df['yield_tonnes_ha'])
df['log_hectares'] = np.log1p(df['hectares_ha'])
df['log_production'] = np.log1p(df['production_tonnes'])

In [7]:
new_order = ['yield_tonnes_ha', 'national', 'subnational', 'crop', 'hectares_ha', 'log_hectares', 'production_tonnes', 'log_production', 'log_yield']
df = df[new_order]

In [8]:
#data sample
df.shape
df.head(20)

,yield_tonnes_ha,national,subnational,crop,hectares_ha,log_hectares,production_tonnes,log_production,log_yield
59,2.580000,Austria,Austria,wheat,275822.0,12.527515,7.116208e+05,13.475302,1.275363
60,2.612000,Austria,Austria,wheat,270351.0,12.507480,7.061568e+05,13.467594,1.284262
61,2.510000,Austria,Austria,wheat,274722.0,12.523519,6.895522e+05,13.443799,1.255616
62,2.652000,Austria,Austria,wheat,282967.0,12.553089,7.504285e+05,13.528401,1.295275
63,2.397000,Austria,Austria,wheat,275654.0,12.526905,6.607426e+05,13.401121,1.222893
64,2.858000,Austria,Austria,wheat,313812.0,12.656553,8.968747e+05,13.706673,1.350149
65,3.305000,Austria,Austria,wheat,316319.0,12.664510,1.045434e+06,13.859944,1.459777
66,3.417000,Austria,Austria,wheat,305691.0,12.630333,1.044546e+06,13.859094,1.485461
67,3.317000,Austria,Austria,wheat,286470.0,12.565393,9.502210e+05,13.764451,1.462561
68,2.959999,Austria,Austria,wheat,275229.0,12.525362,8.146775e+05,13.610549,1.376244


In [9]:
#train and validate model
train, test_and_validate = train_test_split(df, 
                                            test_size=0.2, 
                                            random_state=20) 
                                            #stratify=df['yield_tonnes_ha'])

In [10]:
#test and validate model
test, validate = train_test_split(test_and_validate, 
                                  test_size=0.5, 
                                  random_state=20)
                                 #stratify=df['yield(tonnes/ha)'])

In [11]:
print(train.shape)
print(test.shape)
print(validate.shape)

t1 = train['yield_tonnes_ha'].value_counts()
t2 = test['yield_tonnes_ha'].value_counts()
t3 = validate['yield_tonnes_ha'].value_counts()
result = pd.concat([t1,t2,t3], axis=1, sort=False)
result

(27324, 9)
(3415, 9)
(3416, 9)


,count,count,count
yield_tonnes_ha,,,
1.210527,163.0,23.0,19.0
1.277778,163.0,15.0,18.0
1.345030,161.0,22.0,18.0
1.076024,142.0,19.0,26.0
1.143275,139.0,15.0,26.0
...,...,...,...
1.764650,NaN,NaN,1.0
0.924000,NaN,NaN,1.0
2.591171,NaN,NaN,1.0


In [13]:
#upload to s3 bucket
bucket='c93435a2086654l5105130t1w6478590828-sandboxbucket-26l6r8c9sh60'
prefix='mod03-demo-training-a-model'
train_file='crop_train.csv'
test_file='crop_test.csv'
validate_file='crop_validate.csv'
whole_file='crop.csv'
s3_resource = boto3.Session().resource('s3')

def upload_s3_csv(filename, folder, dataframe):
    csv_buffer = io.StringIO()
    dataframe.to_csv(csv_buffer, header=False, index=False )
    s3_resource.Bucket(bucket).Object(os.path.join(prefix, folder, filename)).put(Body=csv_buffer.getvalue())

upload_s3_csv(train_file, 'train', train)
upload_s3_csv(test_file, 'test', test)
upload_s3_csv(validate_file, 'validate', validate)

In [14]:
role=sagemaker.get_execution_role()
s3_output_location="s3://{}/{}/output/".format(bucket,prefix)
container = retrieve('xgboost',boto3.Session().region_name,'1.0-1')

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [19]:
hyperparams={
    "num_round":"20",
    "num_class":"7",
    "objective":"multi:softmax"}

In [20]:
xgb_model=sagemaker.estimator.Estimator(container,
                                        role,
                                        instance_count=1,
                                        instance_type='ml.m4.xlarge',
                                        output_path=s3_output_location,
                                        hyperparameters=hyperparams,
                                        sagemaker_session=sagemaker.Session())

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [21]:
train_channel = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/train/".format(bucket,prefix,train_file),
    content_type='text/csv')

validate_channel = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/validate/".format(bucket,prefix,validate_file),
    content_type='text/csv')

data_channels = {'train': train_channel, 'validation': validate_channel}

In [26]:
#input channels
#xgb_model.fit(inputs=data_channels, logs=False)
xgb_model.fit(t1[['yield_tonnes_ha', 'production_tonnes']], t1['yield_tonnes_ha'])

KeyError: "None of [Index(['yield_tonnes_ha', 'production_tonnes'], dtype='object', name='yield_tonnes_ha')] are in the [index]"

In [ ]:
s=sagemaker.analytics.TrainingJobAnalytics(xgb_model._current_job_name, 
                                         metric_names = ['train:merror', 
                                                         'validation:merror']
                                        )

s_df=s.dataframe()
s_df = s_df.iloc[:,1:12]
s_df
#(wrong cases)/#(all cases)